# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Aug  5 2022  3:42PM  243601        10  0085925603  ISDIN Corporation   
1  Aug  5 2022  3:42PM  243601        10  0085925602  ISDIN Corporation   
2  Aug  5 2022  3:42PM  243601        10  0085925604  ISDIN Corporation   
3  Aug  5 2022  3:42PM  243601        10  0085925608  ISDIN Corporation   
4  Aug  5 2022  3:42PM  243601        10  0085925607  ISDIN Corporation   
5  Aug  5 2022  3:40PM  243600        22      623224  NBTY Global, Inc.   
6  Aug  5 2022  3:20PM  243592        10  0085925305  ISDIN Corporation   
7  Aug  5 2022  3:20PM  243592        10  0085925306  ISDIN Corporation   
8  Aug  5 2022  3:20PM  243592        10  0085925359  ISDIN Corporation   
9  Aug  5 2022  3:20PM  243592        10  0085925360  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
39  243594       Released          1
40  243595      Executing          1
41  243596       Released          1
42  243600       Released          1
43  243601       Released          5

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
243594                NaN        NaN       1.0
243595                NaN        1.0       NaN
243596                NaN        NaN       1.0
243600                NaN        NaN       1.0
243601                NaN        NaN       5.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
243508                0.0        0.0       1.0
243509                0.0        0.0       1.0
243517                0.0        0.0       1.0
243532                0.0        1.0       0.0
243533                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
243508                  0          0         1
243509                  0          0         1
243517                  0          0         1
243532                  0          1         0
243533                  0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               243508          0          0         1
1               243509          0          0         1
2               243517          0          0         1
3               243532          0          1         0
4               243533          0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               243508                            1
1               243509                            1
2               243517                            1
3               243532                   1         
4               243533                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                 Customer
0   Aug  5 2022  3:42PM  243601        10        ISDIN Corporation
5   Aug  5 2022  3:40PM  243600        22        NBTY Global, Inc.
6   Aug  5 2022  3:20PM  243592        10        ISDIN Corporation
52  Aug  5 2022  3:19PM  243596        22        NBTY Global, Inc.
53  Aug  5 2022  3:18PM  243595        22        NBTY Global, Inc.
54  Aug  5 2022  3:18PM  243594        10              Emerginnova
55  Aug  5 2022  2:54PM  243593        10                  Bio-PRF
57  Aug  5 2022  2:46PM  243590        10        ISDIN Corporation
63  Aug  5 2022  2:46PM  243591        10           Citieffe, Inc.
65  Aug  5 2022  2:44PM  243589        19  GUSA Granules USA, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                 Customer Completed  \
0  Aug  5 2022  3:42PM  243601        10        ISDIN Corporation             
1  Aug  5 2022  3:40PM  243600        22        NBTY Global, Inc.             
2  Aug  5 2022  3:20PM  243592        10        ISDIN Corporation             
3  Aug  5 2022  3:19PM  243596        22        NBTY Global, Inc.             
4  Aug  5 2022  3:18PM  243595        22        NBTY Global, Inc.             
5  Aug  5 2022  3:18PM  243594        10              Emerginnova             
6  Aug  5 2022  2:54PM  243593        10                  Bio-PRF             
7  Aug  5 2022  2:46PM  243590        10        ISDIN Corporation             
8  Aug  5 2022  2:46PM  243591        10           Citieffe, Inc.             
9  Aug  5 2022  2:44PM  243589        19  GUSA Granules USA, Inc.             

  Executing Released  
0                  5  
1                  1  
2                 46  
3                  1  
4         1           
5                  1  
6                  2  
7                  6  
8                  2  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                 Customer Released  \
0  Aug  5 2022  3:42PM  243601        10        ISDIN Corporation        5   
1  Aug  5 2022  3:40PM  243600        22        NBTY Global, Inc.        1   
2  Aug  5 2022  3:20PM  243592        10        ISDIN Corporation       46   
3  Aug  5 2022  3:19PM  243596        22        NBTY Global, Inc.        1   
4  Aug  5 2022  3:18PM  243595        22        NBTY Global, Inc.            
5  Aug  5 2022  3:18PM  243594        10              Emerginnova        1   
6  Aug  5 2022  2:54PM  243593        10                  Bio-PRF        2   
7  Aug  5 2022  2:46PM  243590        10        ISDIN Corporation        6   
8  Aug  5 2022  2:46PM  243591        10           Citieffe, Inc.        2   
9  Aug  5 2022  2:44PM  243589        19  GUSA Granules USA, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4         1            
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse           Customer Released  \
0  Aug  5 2022  3:42PM  243601        10  ISDIN Corporation        5   
1  Aug  5 2022  3:40PM  243600        22  NBTY Global, Inc.        1   
2  Aug  5 2022  3:20PM  243592        10  ISDIN Corporation       46   
3  Aug  5 2022  3:19PM  243596        22  NBTY Global, Inc.        1   
4  Aug  5 2022  3:18PM  243595        22  NBTY Global, Inc.            

  Executing Completed  
0                      
1                      
2                      
3                      
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Aug  5 2022  3:42PM  243601        10  ISDIN Corporation       5.0   
1  Aug  5 2022  3:40PM  243600        22  NBTY Global, Inc.       1.0   
2  Aug  5 2022  3:20PM  243592        10  ISDIN Corporation      46.0   
3  Aug  5 2022  3:19PM  243596        22  NBTY Global, Inc.       1.0   
4  Aug  5 2022  3:18PM  243595        22  NBTY Global, Inc.       NaN   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse           Customer  Released  \
0  Aug  5 2022  3:42PM  243601        10  ISDIN Corporation       5.0   
1  Aug  5 2022  3:40PM  243600        22  NBTY Global, Inc.       1.0   
2  Aug  5 2022  3:20PM  243592        10  ISDIN Corporation      46.0   
3  Aug  5 2022  3:19PM  243596        22  NBTY Global, Inc.       1.0   
4  Aug  5 2022  3:18PM  243595        22  NBTY Global, Inc.       0.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3653704      98.0        0.0        0.0
15          243509       1.0        0.0        0.0
16         2922580      13.0        3.0        0.0
19         1948537       9.0        3.0        9.0
20          487117      10.0       21.0        0.0
22          730791       2.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3653704      98.0        0.0        0.0
1        15   243509       1.0        0.0        0.0
2        16  2922580      13.0        3.0        0.0
3        19  1948537       9.0        3.0        9.0
4        20   487117      10.0       21.0        0.0
5        22   730791       2.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      98.0        0.0        0.0
1        15       1.0        0.0        0.0
2        16      13.0        3.0        0.0
3        19       9.0        3.0        9.0
4        20      10.0       21.0        0.0
5        22       2.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   98.0
1        15  Released    1.0
2        16  Released   13.0
3        19  Released    9.0
4        20  Released   10.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   15    16   19    20   22
Status                                    
Completed   0.0  0.0   0.0  9.0   0.0  0.0
Executing   0.0  0.0   3.0  3.0  21.0  1.0
Released   98.0  1.0  13.0  9.0  10.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   15    16   19    20   22
0          Completed   0.0  0.0   0.0  9.0   0.0  0.0
1          Executing   0.0  0.0   3.0  3.0  21.0  1.0
2           Released  98.0  1.0  13.0  9.0  10.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   15    16   19    20   22
0  Completed   0.0  0.0   0.0  9.0   0.0  0.0
1  Executing   0.0  0.0   3.0  3.0  21.0  1.0
2   Released  98.0  1.0  13.0  9.0  10.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()